In [1]:
import h5py
import numpy as np
import torch.nn as nn
import torch
import sys
from torchsummary import summary
# from torchvision import transforms, datasets, utils
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import torch.optim as optim
from tqdm import tqdm
import math
import data_process as dp
%matplotlib inline

In [2]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [3]:
file = h5py.File('image_norm_12000.h5','r')

In [4]:
imageData = file['imageData'][:]
imageLabel = file['imageLabel'][:]
file.close()

In [5]:
#切割数据集，切割为原始数据集的5/6
def Cut_Data(data):
    N = data.shape[0]
    new_d = []
    for i in range(0,N):
        if i%6 == 0:
            continue
        new_d.append(data[i])
    return np.array(new_d)

In [6]:
#随机打乱标签和数据
def Disorder_Data(data,label):
    N = data.shape[0]
    index = np.random.permutation(N)
    new_data = data[index]
    new_label = label[index]
    return np.array(new_data),np.array(new_label)

In [7]:
l1,l2,d1,d2 = [],[],[],[]
for i in range(10):
    start = i * 200
    end = (i+1) * 200
    data_split = imageData[start:end]
    label_split = imageLabel[start:end]
    N = data_split.shape[0]
    index = np.random.permutation(N)
    data = data_split[index,:,:]
    label = label_split[index]
    num_train = round(N*0.85)
    d1.append(data[0:num_train,:,:])
    d2.append(data[num_train:N,:,:])
    l1.append(label[0:num_train])
    l2.append(label[num_train:N])
ptrain_data = np.concatenate([d1[0],d1[1],d1[2],d1[3]],axis=0)
ptrain_label = np.concatenate([l1[0],l1[1],l1[2],l1[3]],axis=0)
ptest_data = np.concatenate([d2[0],d2[1],d2[2],d2[3]],axis=0)
ptest_label = np.concatenate([l2[0],l2[1],l2[2],l2[3]],axis=0)

In [8]:
#再次打乱数据和标签
ptrain_data,ptrain_label  = Disorder_Data(ptrain_data,ptrain_label)
ptest_data,ptest_label = Disorder_Data(ptest_data,ptest_label)


分割数据,同时升维

In [12]:
N = ptrain_data.shape[0]
num_split = 200
for i in range(N):
    new = ptrain_data[i]
    rece = Cut_Data(new)
    print(rece.shape[0]/200)
rece.shape

50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0


(10000, 4)

In [53]:
def Split_Data(data):
    N = data.shape[0]
    print(N)
    d1 = []
    num_split = 200
    for i in range(0,N):
        new_l = []
        new = data[i]
        Count = int(data.shape[1] / num_split)
        for j in range(Count):
            start = j * num_split
            end = (j + 1) * num_split
            new_l.append(new[start:end])
        d1.append(np.array(new_l))
    N = len(d1)
    train_d = d1[0]
    for i in range(1,N):
        train_d = np.concatenate([train_d,d1[i]],axis=0)
    train_d  = np.expand_dims(train_d, axis=3)
    return train_d

分割标签

In [54]:
def Split_Label(labels):
    N = labels.shape[0]
    l1 = []
    for i in range(N):
        num = labels[i]
        new_l = np.zeros(60,dtype=int)
        new_l[:] = num
        l1.append(new_l)
    N = len(l1)
    train_d = l1[0]
    for i in range(1,N):
        train_d = np.concatenate([train_d,l1[i]],axis=0)
    train_d = convert_to_one_hot(train_d,10).T
    return train_d

In [55]:
# #分割数据，同时对数据进行升维
# d1,d2,l1,l2 = [],[],[],[]
# N = ptrain_data.shape[0]
# print(N)
# num_split = 200
# for i in range(0,N):
#     new_l = []
#     new = ptrain_data[i]
#     Count = int(ptrain_data.shape[1] / num_split)
#     for j in range(Count):
#         start = j * num_split
#         end = (j + 1) * num_split
#         new_l.append(new[start:end])
#     d1.append(np.array(new_l))
# N = len(d1)
# train_d = d1[0]
# for i in range(1,N):
#     train_d = np.concatenate([train_d,d1[i]],axis=0)



In [56]:
# file = h5py.File('image_norm_200.h5','r')
# imageData   = file['imageData'][:]
# print(imageData.shape)
# imageData   = imageData*2000
# print(imageData.shape)
# imageLabel  = file['imageLabel'][:]
# file.close()
# # 随机打乱数据和标签
# N = imageData.shape[0]
# index = np.random.permutation(N)
# data  = imageData[index,:,:]
# label = imageLabel[index]
# print(label.shape)
# # 对数据升维,标签one-hot
# print(data.shape)
# data  = np.expand_dims(data, axis=3)
# label = convert_to_one_hot(label,10).T
# print(label.shape)

In [57]:
# def convert_to_one_hot(Y, C):
#     Y = np.eye(C)[Y.reshape(-1)].T
#     return Y

In [58]:
# file = h5py.File('image_norm_200.h5','r')
# imageData   = file['imageData'][:]
# print(imageData.shape)
# imageData   = imageData*2000
# print(imageData.shape)
# imageLabel  = file['imageLabel'][:]
# file.close()
# # 随机打乱数据和标签
# N = imageData.shape[0]
# index = np.random.permutation(N)
# data  = imageData[index,:,:]
# label = imageLabel[index]
# # 对数据升维,标签one-hot
# print(data.shape)
# data  = np.expand_dims(data, axis=3)
# label = convert_to_one_hot(label,10).T

In [59]:
# print(data.shape)
# print(label.shape)

In [60]:
# # 划分数据集
# N = data.shape[0]
# num_train = round(N*0.8)
# X_train = data[0:num_train,:,:,:]
# Y_train = label[0:num_train]
# X_test  = data[num_train:N,:,:,:]
# Y_test  = label[num_train:N]

In [61]:
X_train = Split_Data(ptrain_data)
Y_train = Split_Label(ptrain_label)
X_test = Split_Data(ptest_data)
Y_test = Split_Label(ptest_label)

680
120


In [62]:
X_train = torch.Tensor(X_train).permute(0,3,1,2)
Y_train = torch.Tensor(Y_train)
X_test = torch.Tensor(X_test).permute(0,3,1,2)
Y_test = torch.Tensor(Y_test)

Y_train = torch.max(Y_train, dim=1)[1]
Y_test = torch.max(Y_test, dim=1)[1]

In [63]:
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

X_train shape: torch.Size([40800, 1, 200, 4])
Y_train shape: torch.Size([40800])
X_test shape: torch.Size([7200, 1, 200, 4])
Y_test shape: torch.Size([7200])


分割前的数据集：
680
(161160, 200, 4, 1)
(161160, 10)
120
(28440, 200, 4, 1)
(28440, 10)

分割后：
680
(161160, 200, 4, 1)
(161160, 10)
120
(3600, 200, 4, 1)
(3600, 10)




680
(161160, 200, 4, 1)
(161160, 10)
120
(4440, 200, 4, 1)
(4440, 10)

In [2]:
# train_dataset = TensorDataset(X_train, Y_train)
# validate_dataset = TensorDataset(X_test, Y_test)

train_dataset,validate_dataset = dp.main()


1190
(71400, 200, 4, 1)
(71400, 10)
210
(12600, 200, 4, 1)
(12600, 10)


In [3]:
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=240,
                          shuffle=True)
validate_loader = DataLoader(dataset=validate_dataset,
                          batch_size=240,
                          shuffle=True)

In [4]:
class Block(nn.Module):
    def __init__(self, num_classes):
        super(Block, self).__init__()
        self.features1 = nn.Sequential(
            nn.Conv2d(1, 32,kernel_size=(20,3), stride=(1,1),padding=1),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(20,1)),

            nn.Conv2d(32, 64,kernel_size=(3,1), stride=(1,1), padding=1),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(7,1)),

        )
        self.features2 = nn.Sequential(
            nn.Conv2d(1, 32,kernel_size=(16,3), stride=(1,1),padding=1),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(20,1)),

            nn.Conv2d(32, 64,kernel_size=(4,1), stride=(1,1), padding=1),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(6,1)),
        )
        self.features3 = nn.Sequential(
            nn.Conv2d(1, 32,kernel_size=(12,3), stride=(1,1),padding=1),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(20,1)),

            nn.Conv2d(32, 64,kernel_size=(5,1), stride=(1,1), padding=1),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(5,1)),
        )
        self.features4 = nn.Sequential(
            nn.Conv2d(1, 32,kernel_size=(8,3), stride=(1,1),padding=1),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(20,1)),

            nn.Conv2d(32, 64,kernel_size=(6,1), stride=(1,1), padding=1),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(5,1)),
        )

        self.features5 = nn.Sequential(
            nn.Conv2d(256, 32,kernel_size=(3,1), stride=(1,1),padding=1),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(20,1)),

            nn.Conv2d(32, 64,kernel_size=(4,1), stride=(1,1), padding=1),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(6,1)),
        )


        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(1536, 512),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def multi_block(self, x):
        x1 = self.features1(x)
        x2 = self.features2(x)
        x3 = self.features3(x)
        x4 = self.features4(x)

        x = torch.concat((x1, x2, x3, x4), 1)
        # print(x.size())
        x = torch.flatten(x, start_dim=1)
        


        # x = torch.flatten(x, start_dim=1)
        return x

    def forward(self, x):
        x = self.multi_block(x)
        
        x = self.classifier(x)
        # print(x.size())
        return x

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = Block(num_classes=10)
net.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.005)
epochs = 30
best_acc = 0.0
train_steps = len(train_loader)
val_num = len(validate_dataset)

In [6]:
summary(net, (1,200,4))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 183, 4]           1,952
         LeakyReLU-2           [-1, 32, 183, 4]               0
         MaxPool2d-3             [-1, 32, 9, 4]               0
            Conv2d-4             [-1, 64, 9, 6]           6,208
         LeakyReLU-5             [-1, 64, 9, 6]               0
         MaxPool2d-6             [-1, 64, 1, 6]               0
            Conv2d-7           [-1, 32, 187, 4]           1,568
         LeakyReLU-8           [-1, 32, 187, 4]               0
         MaxPool2d-9             [-1, 32, 9, 4]               0
           Conv2d-10             [-1, 64, 8, 6]           8,256
        LeakyReLU-11             [-1, 64, 8, 6]               0
        MaxPool2d-12             [-1, 64, 1, 6]               0
           Conv2d-13           [-1, 32, 191, 4]           1,184
        LeakyReLU-14           [-1, 32,

In [7]:
print("using {} device.".format(device))
train_loss = []
val_acc = []
for epoch in range(epochs):
    # train
    net.train()
    running_loss = 0.0
    train_bar = tqdm(train_loader, file=sys.stdout)
    
    for step, data in enumerate(train_bar):
        train_data, train_labels = data
        optimizer.zero_grad()
        outputs = net(train_data.to(device))
        loss = loss_function(outputs, train_labels.to(device))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        train_loss.append(loss)
        train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1,epochs,loss)

    # validate
    net.eval()
    acc = 0.0  # accumulate accurate number / epoch
    with torch.no_grad():
        val_bar = tqdm(validate_loader, file=sys.stdout)
        for data in val_bar:
            val_data, val_labels = data
            outputs = net(val_data.to(device))
            predict_y = torch.max(outputs, dim=1)[1]

            acc += torch.eq(predict_y, val_labels.to(device)).sum().item()
            val_acc.append(acc)

    val_accurate = acc / val_num
    print('[epoch %d] train_loss: %.3f  val_accuracy: %.3f' %
          (epoch + 1, running_loss / train_steps, val_accurate))

print('Finished Training')

using cuda:0 device.
100%|██████████| 53/53 [00:00<00:00, 237.24it/s]
[epoch 1] train_loss: 1.099  val_accuracy: 0.650
100%|██████████| 53/53 [00:00<00:00, 234.10it/s]
[epoch 2] train_loss: 0.914  val_accuracy: 0.661
100%|██████████| 53/53 [00:00<00:00, 250.67it/s]
[epoch 3] train_loss: 0.879  val_accuracy: 0.678
100%|██████████| 53/53 [00:00<00:00, 253.06it/s]
[epoch 4] train_loss: 0.861  val_accuracy: 0.685
100%|██████████| 53/53 [00:00<00:00, 254.27it/s]
[epoch 5] train_loss: 0.843  val_accuracy: 0.683
100%|██████████| 53/53 [00:00<00:00, 251.86it/s]
[epoch 6] train_loss: 0.831  val_accuracy: 0.692
100%|██████████| 53/53 [00:00<00:00, 240.46it/s]
[epoch 7] train_loss: 0.819  val_accuracy: 0.687
100%|██████████| 53/53 [00:00<00:00, 254.27it/s]
[epoch 8] train_loss: 0.809  val_accuracy: 0.692
100%|██████████| 53/53 [00:00<00:00, 260.50it/s]
[epoch 9] train_loss: 0.802  val_accuracy: 0.690
100%|██████████| 53/53 [00:00<00:00, 253.06it/s]
[epoch 10] train_loss: 0.799  val_accuracy: 0.68

KeyboardInterrupt: 

In [ ]:
train_loss[:][0].item()
x1 = []
for each in train_loss:
    x1.append(each.item())
x2 = np.divide(val_acc, val_num)


In [ ]:
plt.figure(figsize=(50, 16), dpi=500)
plt.subplot(2, 1, 1)
plt.xlabel("train_loss")
# plt.plot(x1, color="red", linestyle="--")
plt.scatter(range(len(x1)),x1)
plt.subplot(2, 1, 2)
plt.xlabel("val_acc")
# plt.plot(x2, color="blue", linestyle="--")
plt.scatter(range(len(x2)),x2)

plt.show()

In [ ]:
# class Block(nn.Module):
#     def __init__(self, num_classes):
#         super(Block, self).__init__()
#         self.features1 = nn.Sequential(
#             nn.Conv2d(1, 32,kernel_size=(20,3), stride=(1,1),padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=(20,1)),

#             nn.Conv2d(32, 64,kernel_size=(3,1), stride=(1,1), padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=(7,1)),
#         )
#         self.features2 = nn.Sequential(
#             nn.Conv2d(1, 32,kernel_size=(16,3), stride=(1,1),padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=(20,1)),

#             nn.Conv2d(32, 64,kernel_size=(4,1), stride=(1,1), padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=(6,1)),
#         )
#         self.features3 = nn.Sequential(
#             nn.Conv2d(1, 32,kernel_size=(12,3), stride=(1,1),padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=(20,1)),

#             nn.Conv2d(32, 64,kernel_size=(5,1), stride=(1,1), padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=(5,1)),
#         )
#         self.features4 = nn.Sequential(
#             nn.Conv2d(1, 32,kernel_size=(8,3), stride=(1,1),padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=(20,1)),

#             nn.Conv2d(32, 64,kernel_size=(6,1), stride=(1,1), padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=(5,1)),
#         )

#         self.classifier = nn.Sequential(
#             nn.Dropout(0.5),
#             nn.Linear(1536, 128),
#             nn.ReLU(inplace=True),
#             nn.Dropout(0.5),
#             nn.Linear(128, num_classes)
#         )
#     def forward(self, x):
#         x1 = self.features1(x)
#         x2 = self.features2(x)
#         x3 = self.features3(x)
#         x4 = self.features4(x)
        
#         x = torch.concat((x1, x2, x3, x4), 1)
#         # print(x.size())
#         x = torch.flatten(x, start_dim=1)
#         x = self.classifier(x)
#         # print(x.size())
#         return x

In [ ]:
# import numpy as np

In [ ]:
# a = []
# for i in range(3):
#     a.append(np.zeros((3,2)))
# print(np.concatenate(a, axis=0).shape)
# print(a[1])